In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import re
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import numpy as np
import os



In [ ]:
path = '/gpfs/data/ssa/users/d602145/Workspace/scratch/Porosity/ETH/'
os.chdir(path)

In [ ]:
from Lib.Data import PorosityDistribution, Extract_data
from Lib.Datasets import  PorosityDataset

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

In [ ]:
sample_path = os.getcwd()+'/Job_Assignment_Data/Job_Assignment_Data/'

In [ ]:
extracted_data = Extract_data(sample_path,keep_density_doubles=False)

In [ ]:
extracted_data[77].plot_porosity_distribution()

In [ ]:
len(extracted_data.keys())

In [ ]:
# Create train, validation, and test datasets
train_dataset = PorosityDataset(sample_path, train=True, val=False, test=False,keep_doubles=False)
val_dataset = PorosityDataset(sample_path, train=False, val=True, test=False,keep_doubles=False)
test_dataset = PorosityDataset(sample_path, train=False, val=False, test=True,keep_doubles=False)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self,scale=1):
        super(Encoder, self).__init__()
        self.scale = scale

        # Density Condition
        self.fc1c = nn.Sequential(nn.Linear(1, 32 * scale), nn.Dropout(0.1), nn.SiLU())
        self.fc2c = nn.Sequential(nn.Linear(1, 16 * scale), nn.Dropout(0.1), nn.SiLU())


        # 3D Convolutional Layers with Batch Normalization
        self.conv1 = nn.Conv3d(1, 8*scale, kernel_size=3, stride=2) #Bx8x14x14x14
        self.bn1 = nn.BatchNorm3d(8*scale)  # Batch Normalization after conv1
        self.conv2 = nn.Conv3d(8*scale, 16*scale, kernel_size=3, stride=2) #Bx16x6x6x6
        self.bn2 = nn.BatchNorm3d(16*scale)  # Batch Normalization after conv2
        self.conv3 = nn.Conv3d(16*scale, 32*scale, kernel_size=3, stride=2) #Bx32x2x2x2
        self.bn3 = nn.BatchNorm3d(32*scale)  # Batch Normalization after conv3
        self.conv4 = nn.Conv3d(32*scale, 64*scale, kernel_size=2, stride=2) #Bx64x1x1x1
        self.bn4 = nn.BatchNorm3d(64*scale)  # Batch Normalization after conv3

        # Linear Layers with Dropout
        self.fc1 = nn.Linear(64*scale, 32*scale)
        self.dropout1 = nn.Dropout(0.1)  # Dropout after fc1
        self.fc2 = nn.Linear(32*scale, 16*scale)
        self.dropout2 = nn.Dropout(0.1)  # Dropout after fc2
        self.fc3 = nn.Linear(16*scale, 8*scale)

        # Activation Function
        self.Silu = nn.SiLU()

    def convolution(self,x):
        x = self.Silu(self.bn1(self.conv1(x)))
        x = self.Silu(self.bn2(self.conv2(x)))
        x = self.Silu(self.bn3(self.conv3(x)))
        x = self.Silu(self.bn4(self.conv4(x)))

        return x

    def nl_projection(self,x,y):
        x = self.Silu(self.dropout1(self.fc1(x)))
        x = x + self.fc1c(y.view(-1, 1))
        x = self.Silu(self.dropout2(self.fc2(x)))
        x = x + self.fc2c(y.view(-1, 1))
        x = self.fc3(x)

        return x

    def forward(self, x, y):
        x = self.convolution(x)
        x = x.view(-1, 64 * self.scale)  # Flatten for linear layers
        x = self.nl_projection(x,y)

        return x.squeeze()

class Decoder2(nn.Module):
    def __init__(self, scale=1):
        super(Decoder2, self).__init__()
        self.scale = scale

        # Density Condition
        self.fc1c = nn.Sequential(nn.Linear(1, 16 * scale), nn.Dropout(0.1), nn.SiLU())
        self.fc2c = nn.Sequential(nn.Linear(1, 32 * scale), nn.Dropout(0.1), nn.SiLU())

        # Linear Layers with Dropout (mirroring encoder)

        #input Bx8

        self.fc1 = nn.Linear(8 * scale, 16 * scale)
        self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(16 * scale, 32 * scale)
        self.dropout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(32 * scale, 64 * scale)
        self.dropout3 = nn.Dropout(0.1)
        self.fc4 = nn.Linear(64 * scale, 27000)

        # 3D Convolutional Transpose Layers with Batch Normalization

        #input: Bx64x1x1x1

        # Activation Function
        self.Silu = nn.SiLU()

    def nl_projection(self, x, y):
        x = self.Silu(self.dropout1(self.fc1(x)))
        x = x + self.fc1c(y.view(-1, 1))
        x = self.Silu(self.dropout2(self.fc2(x)))
        x = x + self.fc2c(y.view(-1, 1))
        x = self.Silu(self.fc3(x))
        x = self.fc4(x)
        return x

    def forward(self, x, y):
        x = self.nl_projection(x,y)
        x = x.view(-1, 1, 30, 30, 30)
        return torch.sigmoid(x)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, scale=1):
        super(Decoder, self).__init__()
        self.scale = scale

        # Density Condition
        self.fc1c = nn.Sequential(nn.Linear(1, 16 * scale), nn.Dropout(0.1), nn.SiLU())
        self.fc2c = nn.Sequential(nn.Linear(1, 32 * scale), nn.Dropout(0.1), nn.SiLU())

        # Linear Layers with Dropout (mirroring encoder)

        #input Bx8

        self.fc1 = nn.Linear(8 * scale, 16 * scale)
        self.dropout1 = nn.Dropout(0.1)
        self.fc2 = nn.Linear(16 * scale, 32 * scale)
        self.dropout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(32 * scale, 64 * scale)

        # 3D Convolutional Transpose Layers with Batch Normalization

        #input: Bx64x1x1x1

        self.convT1 = nn.ConvTranspose3d(64 * scale, 32 * scale, kernel_size=2, stride=1) #Bx32x2x2x2
        self.bn1 = nn.BatchNorm3d(32 * scale)
        self.convT2 = nn.ConvTranspose3d(32 * scale, 16 * scale, kernel_size=3, stride=2, output_padding=1) #Bx16x6x6x6
        self.bn2 = nn.BatchNorm3d(16 * scale)
        self.convT3 = nn.ConvTranspose3d(16 * scale, 8 * scale, kernel_size=4, stride=2, output_padding=1) #Bx8x15x15x15
        self.bn3 = nn.BatchNorm3d(8 * scale)
        self.convT4 = nn.ConvTranspose3d(8 * scale, 1, kernel_size=4, stride=2, padding=1) #Bx1x30x30x30

        # Activation Function
        self.Silu = nn.SiLU()

    def nl_projection(self, x, y):
        x = self.Silu(self.dropout1(self.fc1(x)))
        x = x + self.fc1c(y.view(-1, 1))
        x = self.Silu(self.dropout2(self.fc2(x)))
        x = x + self.fc2c(y.view(-1, 1))
        x = self.Silu(self.fc3(x))
        return x

    def convolution(self, x):
        x = self.Silu(self.bn1(self.convT1(x)))
        x = self.Silu(self.bn2(self.convT2(x)))
        x = self.Silu(self.bn3(self.convT3(x)))
        x = torch.sigmoid(self.convT4(x))  # Sigmoid activation for output
        return x

    def forward(self, x, y):
        x = self.nl_projection(x,y)
        x = x.view(-1, 64 * self.scale, 1, 1, 1)  # Reshape for convolutional layers
        x = self.convolution(x)
        return x

In [ ]:
class ConditionedAutoEncoder(nn.Module):
    def __init__(self,scale=1):
        super(ConditionedAutoEncoder, self).__init__()
        self.scale = scale
        self.encoder = Encoder(scale=scale)
        self.decoder = Decoder2(scale=scale)
        self.regressor = nn.Linear(8*scale,1)

    def forward(self, x, y):

        latent = self.encoder(x,y)
        x = self.decoder(latent,y)
        y_rec = self.regressor(latent)
        return x,y_rec.squeeze()



In [ ]:
import torch.nn as nn

class ConditionedVAE(nn.Module):
    def __init__(self, scale=1):
        super(ConditionedVAE, self).__init__()
        self.scale = scale
        self.encoder = Encoder(scale=scale)  # Your existing Encoder
        self.decoder = Decoder2(scale=scale)  # Your existing Decoder

        # Add layers for mean and variance of the latent space
        self.fc_mu = nn.Linear(8 * scale, 8 * scale)  # Output dimension for mean
        self.fc_logvar = nn.Linear(8 * scale, 8 * scale)  # Output dimension for log variance
        self.regressor = nn.Linear(8 * scale, 1)


    def reparameterize(self, mu, logvar):
        """Reparameterization trick to sample from the latent space."""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, x, y):

        # Encode the input
        h = self.encoder(x, y)

        # Get mean and log variance
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)

        # Sample from the latent space
        z = self.reparameterize(mu, logvar)

        # Decode the latent representation
        x_recon = self.decoder(z, y)
        y_recon = self.regressor(z)

        return x_recon, y_recon.squeeze(), mu, logvar

In [ ]:
(X,y) = next(iter(train_dataloader))
model = ConditionedVAE(scale=8)
model(X[:,3,:,:,:].unsqueeze(1),y)

In [ ]:
import torch.optim as optim

# Define the optimizer

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
# Define the loss function
criterion_reconstruction = nn.BCELoss()
criterion_condition = nn.MSELoss()

In [ ]:
# Training loop
num_epochs = 1

train_losses = []
train_recon_losses = []
train_cond_losses = []
train_kl_losses = []
val_losses = []
val_recon_losses = []
val_cond_losses = []
val_kl_losses = []
alpha = 0.9
beta = 0.5

for epoch in range(num_epochs):
    # Training
    model.train()
    running_train_loss = 0.0
    running_train_recon_loss = 0.0
    running_train_cond_loss = 0.0
    running_train_recon_loss = 0.0
    running_train_kl_loss = 0.0

    for i, (inputs, conditions) in enumerate(train_dataloader):
        inputs = inputs[:,3,:,:,:].unsqueeze(1)
        optimizer.zero_grad()
        outputs, reconstructed_conditions, mu, logvar = model(inputs, conditions)
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        # Calculate individual losses
        loss_reconstruction = criterion_reconstruction(outputs, inputs)
        loss_condition = criterion_condition(reconstructed_conditions, conditions)

        # Combine losses with weights (adjust as needed)
        loss = alpha*(loss_reconstruction + beta*loss_condition) + (1-alpha)*kl_loss # Example: 0.1 weight for condition loss

        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        running_train_recon_loss += loss_reconstruction.item()
        running_train_cond_loss += loss_condition.item()
        running_train_kl_loss += kl_loss.item()


    epoch_train_loss = running_train_loss / len(train_dataloader)
    epoch_train_recon_loss = running_train_recon_loss / len(train_dataloader)
    epoch_train_cond_loss = running_train_cond_loss / len(train_dataloader)
    epoch_train_kl_loss = running_train_kl_loss / len(train_dataloader)

    # Validation
    model.eval()
    running_val_loss = 0.0
    running_val_recon_loss = 0.0
    running_val_cond_loss = 0.0
    running_val_kl_loss = 0.0

    with torch.no_grad():
        for i, (inputs, conditions) in enumerate(val_dataloader):
            inputs = inputs[:,3,:,:,:].unsqueeze(1)
            outputs, reconstructed_conditions, mu, logvar = model(inputs, conditions)

            # Calculate individual losses
            loss_reconstruction = criterion_reconstruction(outputs, inputs)
            loss_condition = criterion_condition(reconstructed_conditions, conditions)
            kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

            # Combine losses with weights (adjust as needed)
            loss = alpha*(loss_reconstruction + beta*loss_condition) + (1-alpha)*kl_loss # Example: 0.1 weight for condition loss

            running_val_loss += loss.item()
            running_val_recon_loss += loss_reconstruction.item()
            running_val_cond_loss += loss_condition.item()
            running_val_kl_loss += kl_loss.item()

    epoch_val_loss = running_val_loss / len(val_dataloader)
    epoch_val_recon_loss = running_val_recon_loss / len(val_dataloader)
    epoch_val_cond_loss = running_val_cond_loss / len(val_dataloader)
    epoch_val_kl_loss = running_val_kl_loss / len(val_dataloader)

    print(f"Epoch [{epoch + 1}/{num_epochs}] "
          f"Train Loss: {epoch_train_loss:.4f} "
          f"Train Reconstruction Loss: {epoch_train_recon_loss:.4f} "
          f"Train Condition Loss: {epoch_train_cond_loss:.4f} "
          f"Val Loss: {epoch_val_loss:.4f} "
          f"Val Reconstruction Loss: {epoch_val_recon_loss:.4f} "
          f"Val Condition Loss: {epoch_val_cond_loss:.4f}")

    train_losses.append(epoch_train_loss)
    train_recon_losses.append(epoch_train_recon_loss)
    train_cond_losses.append(epoch_train_cond_loss)
    train_kl_losses.append(epoch_train_kl_loss)
    val_losses.append(epoch_val_loss)
    val_recon_losses.append(epoch_val_recon_loss)
    val_cond_losses.append(epoch_val_cond_loss)
    val_kl_losses.append(epoch_val_kl_loss)


print("Finished Training")

In [ ]:
# Plotting (after the training loop)
plt.figure(figsize=(12, 6))

# Total loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Total Loss')

# Individual losses
plt.subplot(1, 2, 2)
plt.plot(train_recon_losses, label='Train Reconstruction Loss')
plt.plot(train_cond_losses, label='Train Condition Loss')
plt.plot(val_recon_losses, label='Val Reconstruction Loss')
plt.plot(val_cond_losses, label='Val Condition Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Individual Losses')

plt.tight_layout()
plt.show()

In [ ]:
original_data = extracted_data[0]

In [ ]:
original_data.plot_porosity_distribution()

In [ ]:
original_data.plot_porosity_distribution()

In [ ]:
device = next(model.parameters()).device
micro = original_data.as_tensor().to(device)
micro = micro.permute(3,0,1,2)
micro_rec = micro
X,y = micro[3,:,:,:].unsqueeze(0),micro[4,0,0,0]
X_rec, y_rec, mu, logvar = model(X.unsqueeze(0),y)
micro_rec[3,:,:,:] = X_rec.squeeze()
micro_rec[4,:,:,:] = y_rec
micro_rec = micro_rec.permute(1,2,3,0).reshape(30*30*30,-1)
micro_rec = micro_rec.detach().cpu().numpy()
print(micro_rec.shape)

rec_data = PorosityDistribution(micro_rec[:,:-1],y_rec.item())


In [ ]:
original_data.distribution.shape

In [ ]:
rec_data.distribution.shape

In [ ]:
rec_data.plot_porosity_distribution(porosity=0.05)

In [ ]:
        df = rec_data.as_dataframe(porosity=0)
        fig = px.histogram(df.iloc[:,3], facet_col='variable', title=f"Porosity Histogram (Density: {rec_data.density})")
        fig.show()

In [ ]:
df = original_data.as_dataframe(porosity=0)
fig = px.histogram(df.iloc[:,3], facet_col='variable', title=f"Porosity Histogram (Density: {original_data.density})")
fig.show()

In [ ]:
plt.plot(rec_data.distribution[:,1])

In [ ]:
plt.plot(rec_data.distribution[:,2])

In [ ]:
px.histogram(rec_data)